# reading data

In [1]:
import modin.pandas as pd
sales = pd.read_csv("./data/sales_train_validation.csv")
prices =pd.read_csv("./data/sell_prices.csv")
calender =pd.read_csv("./data/calendar.csv")


To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [2]:
print(f" number of colums = {len(sales.columns)}")
print(f" number of rows = {len(sales.id.unique())}")
sales.head()

 number of colums = 1919
 number of rows = 30490


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [3]:

print(f" number of rows = {len(calender.date.unique())}")
calender.head()

 number of rows = 1969


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [4]:
print(f" number of rows = {len(prices.item_id.unique())}")
prices.head()

 number of rows = 3049


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


# preprocessing

In [35]:
from transformer.DataAggregator import DataAggregator

data_aggregator = DataAggregator(sales)
aggregated_data = data_aggregator.aggregate(["store_id","dept_id"],"sum")
aggregated_data=aggregated_data.reset_index()
# aggregated_data.set_index("store_id")

In [36]:
aggregated_data.columns[:3]

Index(['store_id', 'dept_id', 'd_1'], dtype='object')

In [37]:

print(f"number of rows {aggregated_data.d_1.count()}")
aggregated_data.head()

number of rows 70


,store_id,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,CA_1,FOODS_1,297,284,214,175,182,191,224,263,...,293,317,268,302,237,287,297,331,419,329
1,CA_1,FOODS_2,674,655,396,476,354,486,554,747,...,435,499,637,529,391,423,365,489,590,734
2,CA_1,FOODS_3,2268,2198,1398,1607,1496,1730,1915,2952,...,1978,2665,3099,2055,1860,1780,1871,2140,2646,2990
3,CA_1,HOBBIES_1,528,489,409,383,263,453,339,750,...,370,690,597,517,461,333,380,471,621,641
4,CA_1,HOBBIES_2,28,9,6,9,5,14,7,19,...,31,32,58,25,34,30,30,56,49,73


In [21]:
aggregated_data.head(1)

,,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
store_id,dept_id,,,,,,,,,,,,,,,,,,,,,
CA_1,FOODS_1,297,284,214,175,182,191,224,263,245,176,...,293,317,268,302,237,287,297,331,419,329


In [24]:
aggregated_data.transpose()

store_id    CA_1                                                              \
dept_id  FOODS_1 FOODS_2 FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   
d_1          297     674    2268       528        28         361         181   
d_2          284     655    2198       489         9         350         170   
d_3          214     396    1398       409         6         279         114   
d_4          175     476    1607       383         9         278         123   
d_5          182     354    1496       263         5         195         135   
...          ...     ...     ...       ...       ...         ...         ...   
d_1909       287     423    1780       333        30         665         204   
d_1910       297     365    1871       380        30         580         186   
d_1911       331     489    2140       471        56         695         205   
d_1912       419     590    2646       621        49         933         319   
d_1913       329     734    2990       641        73        1022         324   

store_id    CA_2                  ...      WI_2                          \
dept_id  FOODS_1 FOODS_2 FOODS_3  ... HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   
d_1          406     212    1575  ...        10         353          98   
d_2          408     227    1286  ...         7         268          94   
d_3          238     138     913  ...        18         250          69   
d_4          240     174    1126  ...        15         305          80   
d_5          220     102     956  ...         1         118          32   
...          ...     ...     ...  ...       ...         ...         ...   
d_1909       422     450    1634  ...        26         800         113   
d_1910       334     349    1578  ...        25         907         127   
d_1911       579     396    1896  ...        37        1106         160   
d_1912       623     620    2517  ...        25        1177         194   
d_1913       520     782    2710  ...        38        1056         177   

store_id    WI_3                                                              
dept_id  FOODS_1 FOODS_2 FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2  
d_1          152     583    2293       256        22         584         148  
d_2          138     585    2383       342        14         541         195  
d_3          127     575    1841       228        20         420         106  
d_4           98     533    1965       183        11         327          94  
d_5           87     340    1427        70         4         151          53  
...          ...     ...     ...       ...       ...         ...         ...  
d_1909       180     473    1579       202        42         535         148  
d_1910       227     411    1670       182        21         573         142  
d_1911       261     507    1897       264        32         701         166  
d_1912       324     505    2411       381        32         801         232  
d_1913       317     557    2233       230        21         766         201  

[1913 rows x 70 columns]

# rough preprocessing

In [9]:
from transformer.DataAggregator import DataAggregator

data_aggregator = DataAggregator(sales)
aggregated_data = data_aggregator.aggregate(["store_id","dept_id"],"sum")
aggregated_data=aggregated_data.reset_index()
aggregated_data.head()

,store_id,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,CA_1,FOODS_1,297,284,214,175,182,191,224,263,...,293,317,268,302,237,287,297,331,419,329
1,CA_1,FOODS_2,674,655,396,476,354,486,554,747,...,435,499,637,529,391,423,365,489,590,734
2,CA_1,FOODS_3,2268,2198,1398,1607,1496,1730,1915,2952,...,1978,2665,3099,2055,1860,1780,1871,2140,2646,2990
3,CA_1,HOBBIES_1,528,489,409,383,263,453,339,750,...,370,690,597,517,461,333,380,471,621,641
4,CA_1,HOBBIES_2,28,9,6,9,5,14,7,19,...,31,32,58,25,34,30,30,56,49,73


In [10]:
aggregated_data['store_dept_id'] = aggregated_data['store_id'].str.cat(aggregated_data['dept_id'], sep='_')
aggregated_data.drop(columns=['dept_id','store_id'],axis=1, inplace=True) # Drop the unnecessary columns
data = aggregated_data.T  # seting the days as index for new dataframe(Data)
data.columns = aggregated_data['store_dept_id'] # ading product id column to Data 
data.index.name = None   # remove the name of index
data.drop(index='store_dept_id',inplace=True) # remove the first row
# change the format and the freq. of index to datetime format
data.index = calender['date'][0:1913]
# data.index = pd.to_datetime(data.index)
data.index = data.index
data.index.freq= 'd'
data.index

Index(['2011-01-29', '2011-01-30', '2011-01-31', '2011-02-01', '2011-02-02',
       '2011-02-03', '2011-02-04', '2011-02-05', '2011-02-06', '2011-02-07',
       ...
       '2016-04-15', '2016-04-16', '2016-04-17', '2016-04-18', '2016-04-19',
       '2016-04-20', '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24'],
      dtype='object', length=1913)

In [11]:
data.head()

,CA_1_FOODS_1,CA_1_FOODS_2,CA_1_FOODS_3,CA_1_HOBBIES_1,CA_1_HOBBIES_2,CA_1_HOUSEHOLD_1,CA_1_HOUSEHOLD_2,CA_2_FOODS_1,CA_2_FOODS_2,CA_2_FOODS_3,...,WI_2_HOBBIES_2,WI_2_HOUSEHOLD_1,WI_2_HOUSEHOLD_2,WI_3_FOODS_1,WI_3_FOODS_2,WI_3_FOODS_3,WI_3_HOBBIES_1,WI_3_HOBBIES_2,WI_3_HOUSEHOLD_1,WI_3_HOUSEHOLD_2
2011-01-29,297,674,2268,528,28,361,181,406,212,1575,...,10,353,98,152,583,2293,256,22,584,148
2011-01-30,284,655,2198,489,9,350,170,408,227,1286,...,7,268,94,138,585,2383,342,14,541,195
2011-01-31,214,396,1398,409,6,279,114,238,138,913,...,18,250,69,127,575,1841,228,20,420,106
2011-02-01,175,476,1607,383,9,278,123,240,174,1126,...,15,305,80,98,533,1965,183,11,327,94
2011-02-02,182,354,1496,263,5,195,135,220,102,956,...,1,118,32,87,340,1427,70,4,151,53


In [12]:
import pandas 
train = pandas.DataFrame()
train = data['CA_1_FOODS_1']
train=train.reset_index()
train.rename(columns={"index":"ds","CA_1_FOODS_1":"y"},inplace=True)
train.head(2)

,ds,y
0,2011-01-29,297
1,2011-01-30,284


In [13]:
train["ds"].head()

0    2011-01-29
1    2011-01-30
2    2011-01-31
3    2011-02-01
4    2011-02-02
Name: ds, dtype: object

In [14]:
from models.xgboost import XGBoostModel
import numpy as np
xgb = XGBoostModel()
model = xgb.fit(np.asarray(train["ds"]),train["y"])

XGBoostError: [15:20:14] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\data\array_interface.h:475: Unicode-1 is not supported.